In [1]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , CuDNNLSTM
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
import numpy as np
#import tensorflow.compat.v2 as tf

import tensorflow_datasets as tfds


In [ ]:
tf.enable_v2_behavior()

In [ ]:
df_train = tfds.load(name="mnist" , split="train")
df_test = tfds.load(name="mnist" , split="test")
assert isinstance(df_train, tf.data.Dataset)
assert isinstance(df_test, tf.data.Dataset)
print(df_train)

In [ ]:
dataset_length = [i for i,_ in enumerate(df_train)][-1] + 1
print(dataset_length)

In [ ]:
# 28 pixels by each row which are 28. Y hay un total de 60.000 imagenes


In [ ]:
for mnist_example in df_train.take(1):  # Only take a single example
  image, label = mnist_example["image"], mnist_example["label"]

  plt.imshow(image.numpy()[:, :, 0].astype(np.float32), cmap=plt.get_cmap("gray"))
  print("Label: %d" % label.numpy())

In [ ]:
#image.numpy()[: , : , 0].astype(np.float32)
#label.numpy()

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
X_test = mnist.test.images # X_test shape: [num_test, 28*28]
X_test = X_test.reshape([-1, 28, 28])
y_test = mnist.test.labels
X_train= mnist.train.images
X_train = X_train.reshape([-1, 28, 28])
y_train = mnist.train.labels

In [4]:
X_test = X_test / 255.0
X_train = X_train / 255.0

In [5]:
print(X_train.shape[1:])

(28, 28)


### LSTM

In [ ]:
model = Sequential()

model.add(LSTM(128 , input_shape=X_train.shape[1:] , activation='relu' , return_sequences = True ))
model.add(Dropout(0.2))

model.add(LSTM(128 , activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32 , activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(10 , activation='softmax'))

opt = tf.keras.optimizers.Adam(lr = 1e-3 , decay=1e-5)

model.compile(loss = 'sparse_categorical_crossentropy' , optimizer=opt , metrics=['accuracy'])

model.fit(X_train , y_train , epochs=3 , validation_data=(X_test , y_test))

### CuLSTM

In [6]:
model2 = Sequential()

model2.add(CuDNNLSTM(128 , input_shape=X_train.shape[1:] , return_sequences = True ))
model2.add(Dropout(0.2))

model2.add(CuDNNLSTM(128))
model2.add(Dropout(0.2))

model2.add(Dense(32 , activation='relu'))
model2.add(Dropout(0.2))

model2.add(Dense(10 , activation='softmax'))

opt = tf.keras.optimizers.Adam(lr = 1e-3 , decay=1e-5)

model2.compile(loss = 'sparse_categorical_crossentropy' , optimizer=opt , metrics=['accuracy'])

model2.fit(X_train , y_train , epochs=3 , validation_data=(X_test , y_test))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 55000 samples, validate on 10000 samples


InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNNV2' used by node cu_dnnlstm/CudnnRNNV2 (defined at /Users/lucas/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) with these attrs: [seed=0, dropout=0, T=DT_FLOAT, input_mode="linear_input", direction="unidirectional", rnn_mode="lstm", is_training=true, seed2=0]
Registered devices: [CPU, XLA_CPU]
Registered kernels:
  <no registered kernels>

	 [[cu_dnnlstm/CudnnRNNV2]]